In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'fourier'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_fourier_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,984.98,0.03,-0.06,-0.01,0.03,0.00,-0.00,-0.01,0.00,-0.02,...,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.0,-0.00,-0.00
3,0.03,1330.36,0.01,0.00,0.05,0.02,0.01,0.02,0.01,0.01,...,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.0,-0.00,0.00
4,-0.06,0.01,1054.34,-0.02,-0.02,-0.00,0.03,-0.01,0.01,0.01,...,-0.01,0.00,0.00,-0.00,0.00,-0.00,0.00,0.0,0.00,-0.00
5,-0.01,0.00,-0.02,583.07,-0.01,0.01,-0.01,-0.00,-0.01,-0.01,...,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.0,0.00,0.00
6,0.03,0.05,-0.02,-0.01,404.54,-0.00,-0.00,-0.01,-0.01,-0.01,...,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.0,-0.00,-0.00
7,0.00,0.02,-0.00,0.01,-0.00,261.84,-0.00,0.00,0.01,0.00,...,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.0,-0.00,-0.00
8,-0.00,0.01,0.03,-0.01,-0.00,-0.00,195.46,-0.00,0.00,0.01,...,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.0,-0.00,-0.00
9,-0.01,0.02,-0.01,-0.00,-0.01,0.00,-0.00,180.86,-0.00,0.00,...,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.0,0.00,-0.00
10,0.00,0.01,0.01,-0.01,-0.01,0.01,0.00,-0.00,134.00,-0.00,...,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.0,-0.00,0.00
11,-0.02,0.01,0.01,-0.01,-0.01,0.00,0.01,0.00,-0.00,118.51,...,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.0,-0.00,-0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1.00000,0.00003,-0.00006,-0.00002,0.00004,0.00000,-0.00001,-0.00003,0.00001,-0.00005,...,-0.00001,-0.00001,-0.00007,0.00007,0.00003,-0.00000,-0.00001,-0.00006,-0.00002,-0.00000
3,0.00003,1.00000,0.00000,0.00001,0.00006,0.00004,0.00002,0.00004,0.00003,0.00002,...,-0.00004,0.00000,-0.00003,-0.00000,0.00005,0.00002,-0.00001,0.00002,-0.00001,0.00003
4,-0.00006,0.00000,1.00000,-0.00003,-0.00003,-0.00001,0.00006,-0.00002,0.00002,0.00003,...,-0.00007,0.00002,0.00006,-0.00002,0.00001,-0.00006,0.00002,0.00006,0.00002,-0.00001
5,-0.00002,0.00001,-0.00003,1.00000,-0.00003,0.00001,-0.00004,-0.00001,-0.00003,-0.00005,...,0.00003,0.00002,0.00003,0.00000,-0.00001,-0.00002,-0.00001,-0.00002,0.00011,0.00002
6,0.00004,0.00006,-0.00003,-0.00003,1.00000,-0.00001,-0.00001,-0.00003,-0.00003,-0.00002,...,-0.00002,0.00002,0.00004,-0.00004,0.00002,-0.00000,0.00003,-0.00003,-0.00003,-0.00002
7,0.00000,0.00004,-0.00001,0.00001,-0.00001,1.00000,-0.00001,0.00001,0.00004,0.00002,...,0.00001,0.00006,-0.00001,0.00005,-0.00005,0.00002,0.00001,0.00003,-0.00003,-0.00001
8,-0.00001,0.00002,0.00006,-0.00004,-0.00001,-0.00001,1.00000,-0.00002,0.00003,0.00004,...,-0.00002,-0.00004,-0.00000,0.00000,-0.00001,-0.00003,0.00003,-0.00002,-0.00002,-0.00005
9,-0.00003,0.00004,-0.00002,-0.00001,-0.00003,0.00001,-0.00002,1.00000,-0.00001,0.00003,...,0.00003,-0.00001,-0.00004,-0.00001,-0.00001,0.00002,-0.00000,0.00005,0.00002,-0.00001
10,0.00001,0.00003,0.00002,-0.00003,-0.00003,0.00004,0.00003,-0.00001,1.00000,-0.00003,...,0.00002,-0.00000,0.00003,-0.00006,-0.00001,0.00006,0.00001,-0.00003,-0.00005,0.00004
11,-0.00005,0.00002,0.00003,-0.00005,-0.00002,0.00002,0.00004,0.00003,-0.00003,1.00000,...,0.00001,-0.00002,-0.00002,0.00003,-0.00001,0.00003,-0.00002,0.00001,-0.00002,-0.00002


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.17708703350233518

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.54878910e+04 3.49385076e+04 3.00319469e+04 1.05965057e+04
 5.09475654e+03 2.13804014e+03 1.19949560e+03 1.02295636e+03
 8.87545379e+02 5.56507923e+02 4.30334370e+02 2.55883510e+02
 1.95547318e+02 7.69431175e+01 5.05316061e+01 4.34735052e+01
 2.40154459e+01 1.85359717e+01 1.47887390e+01 8.33229800e+00
 6.72162747e+00 5.03025412e+00 2.99885217e+00 1.23228449e+00
 4.84909313e-01 3.38563575e-01 7.01939030e-03 4.33310277e-03
 5.37103305e-04 1.96745178e-04 5.14572376e-06 1.09427556e-26]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.048808,0.996498,-0.064499,-0.015724,-0.009629,-0.005885,-0.004321,-0.003973,-0.002922,-0.002581,...,-0.000226,-0.000485,-0.000089,-0.000080,-0.000010,-0.000009,-0.000003,-0.000002,-3.698654e-07,-9.665238e-08
1,-0.230673,0.051008,0.971331,-0.020690,-0.011615,-0.006799,-0.004890,-0.004536,-0.003304,-0.002903,...,-0.000257,-0.000541,-0.000093,-0.000091,-0.000012,-0.000011,-0.000003,-0.000002,-3.006368e-07,-2.209568e-07
2,0.971009,0.061179,0.226078,-0.038145,-0.020319,-0.011725,-0.008452,-0.007792,-0.005662,-0.005007,...,-0.000431,-0.000930,-0.000170,-0.000149,-0.000020,-0.000018,-0.000006,-0.000005,-7.278482e-07,-3.503845e-07
3,0.030178,0.018125,0.026485,0.997337,-0.048267,-0.019766,-0.013255,-0.012015,-0.008491,-0.007429,...,-0.000612,-0.001338,-0.000237,-0.000219,-0.000029,-0.000026,-0.000009,-0.000006,-3.851357e-07,-3.968328e-07
4,0.017040,0.011580,0.015606,0.045195,0.996884,-0.041310,-0.023003,-0.020380,-0.013517,-0.011627,...,-0.000923,-0.002000,-0.000353,-0.000331,-0.000043,-0.000038,-0.000012,-0.000009,-1.538008e-06,-7.948553e-07
5,0.009446,0.006753,0.008756,0.018456,0.036055,0.994642,-0.060400,-0.046690,-0.023991,-0.019656,...,-0.001347,-0.002933,-0.000525,-0.000474,-0.000066,-0.000055,-0.000018,-0.000012,-2.299974e-06,-1.146302e-06
6,0.005348,0.003883,0.004938,0.009780,0.016192,0.042292,0.970549,-0.204954,-0.035203,-0.025867,...,-0.001411,-0.003105,-0.000545,-0.000499,-0.000068,-0.000059,-0.000019,-0.000014,-2.370525e-06,-1.478267e-06
7,0.005411,0.003915,0.005024,0.009729,0.015778,0.036473,0.154637,0.928541,-0.050100,-0.034402,...,-0.001667,-0.003695,-0.000654,-0.000596,-0.000081,-0.000068,-0.000023,-0.000015,-2.379381e-06,-1.544353e-06
8,0.008680,0.006343,0.008083,0.015522,0.024730,0.053112,0.152535,0.289723,-0.116475,-0.073047,...,-0.003123,-0.006882,-0.001216,-0.001110,-0.000146,-0.000129,-0.000043,-0.000029,-4.760049e-06,-2.334960e-06
9,0.005280,0.003873,0.004922,0.009257,0.014162,0.026060,0.047685,0.059911,0.973017,-0.189356,...,-0.003064,-0.006839,-0.001183,-0.001100,-0.000145,-0.000123,-0.000042,-0.000030,-5.245660e-06,-2.122717e-06


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0488076453472073,
 0.9489920013390984,
 0.7739223524186548,
 0.002662653706427154,
 0.0031160804736105607,
 0.0053575016442403145,
 0.02945066413070596,
 0.07145940360379488,
 1.1164748614921813,
 1.1893562130899524,
 1.067373253240188,
 1.0348876821666644,
 1.04147697791139,
 1.0895480045548653,
 1.041979904302288,
 1.0838168267912478,
 1.0364267215978238,
 1.0625374891008628,
 1.0824536307117152,
 0.995486830819512,
 0.9891311018636343,
 1.0781427996619712,
 0.05730651232227235,
 0.9649191491901843,
 0.14737950076189754,
 0.1552489057223041,
 0.10788438265270783,
 0.1144350545827848,
 0.07591634088198285,
 0.07935524012961559,
 0.11734475312022319,
 0.11636224634606585]